### Notes 

T5 Paper: https://arxiv.org/pdf/1910.10683.pdf

T5 Tokenizer: https://github.com/huggingface/transformers/blob/master/src/transformers/tokenization_t5.py

Important Tasks: https://docs.google.com/document/d/1weIZM6QTlnitpPQmpg-WeV2RW70TnYmDuogBQPr5mB0/edit

In [49]:
#installation step
!pip install transformers
!pip install sentencepiece
#creating the folders 
!mkdir data/
!mkdir data/AD_NMT-master
!mkdir data/train
!mkdir data/test
!mkdir data/val
!mkdir data/vocab
#fetching the pkl files
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1V9crCmqvgQcv0Sx2MCNWB9AET2j6M6FW' -O data/AD_NMT-master/english-Arabic-both.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1V8_tp8ZlWUYaX7QQL46t0uSRNrVehSf1' -O data/AD_NMT-master/english-Arabic-test.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1V7X0qtuDIyjTHY0wh-ZNoVwsiF4lId2e' -O data/AD_NMT-master/english-Arabic-train.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UzL4cOWTMCee83KBUh2QO_H62AFVpDQV' -O data/AD_NMT-master/LAV-MSA-2-both.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UpfCbkxhztof7dvNjeAs1bHjD4SER6h3' -O data/AD_NMT-master/LAV-MSA-2-test.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UlAZGtYsSfXzK7hrC_PbxQFqTSXD0DMw' -O data/AD_NMT-master/LAV-MSA-2-train.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UjDX7cCG2S23SPfSHxSPdVayMTxB5Y16' -O data/AD_NMT-master/Magribi_MSA-both.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UaVWIqRXo0rxuxDF4KArA4bEK1TaLX3l' -O data/AD_NMT-master/Magribi_MSA-test.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UYvlhdYAdfa4riP_4hn3-IEVd1ZUXVTQ' -O data/AD_NMT-master/Magribi_MSA-train.pkl

mkdir: cannot create directory ‘data/’: File exists
mkdir: cannot create directory ‘data/AD_NMT-master’: File exists
mkdir: cannot create directory ‘data/train’: File exists
mkdir: cannot create directory ‘data/test’: File exists
mkdir: cannot create directory ‘data/val’: File exists
mkdir: cannot create directory ‘data/vocab’: File exists
--2020-04-21 19:20:08--  https://docs.google.com/uc?export=download&id=1V9crCmqvgQcv0Sx2MCNWB9AET2j6M6FW
Resolving docs.google.com (docs.google.com)... 172.217.203.101, 172.217.203.100, 172.217.203.138, ...
Connecting to docs.google.com (docs.google.com)|172.217.203.101|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-10-2s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/2fq5m8dvhnn0u0p1m4cf6o38oequlkve/1587496800000/16970776037313924126/*/1V9crCmqvgQcv0Sx2MCNWB9AET2j6M6FW?e=download [following]
--2020-04-21 19:20:08--  https://doc-10-2s-docs.googleusercontent.com/docs/se

In [0]:
#James Chartouni
#Joey Park
#Raef Khan

import torch
import pandas as pd
import numpy as np
import pickle
import os, io, glob

import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

from torchtext.datasets.translation import TranslationDataset
from torchtext.data import Field, BucketIterator

import sentencepiece as spm

from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config, T5PreTrainedModel


from spacy.tokenizer import Tokenizer
from spacy.vocab import Vocab

from sklearn.model_selection import train_test_split


In [51]:
ls data/AD_NMT-master

english-Arabic-both.pkl   LAV-MSA-2-both.pkl   Magribi_MSA-both.pkl
english-Arabic-test.pkl   LAV-MSA-2-test.pkl   Magribi_MSA-test.pkl
english-Arabic-train.pkl  LAV-MSA-2-train.pkl  Magribi_MSA-train.pkl


In [0]:
file_path = 'data/AD_NMT-master/'

with open(file_path + "english-Arabic-train.pkl", 'rb') as handle:
    data_English_MSA_trainval = pickle.load(handle)

with open(file_path + "english-Arabic-test.pkl", 'rb') as handle:
    data_English_MSA_test = pickle.load(handle)

with open(file_path + "english-Arabic-both.pkl", 'rb') as handle:
    data_English_MSA_both = pickle.load(handle) 




with open(file_path + "LAV-MSA-2-train.pkl", 'rb') as handle:
    data_LAV_MSA_trainval = pickle.load(handle) 

with open(file_path + "LAV-MSA-2-test.pkl", 'rb') as handle:
    data_LAV_MSA_test = pickle.load(handle) 

with open(file_path + "LAV-MSA-2-both.pkl", 'rb') as handle:
    data_LAV_MSA_both = pickle.load(handle) 




with open(file_path + "Magribi_MSA-train.pkl", 'rb') as handle:
    data_Magribi_MSA_trainval = pickle.load(handle) 
    
with open(file_path + "Magribi_MSA-test.pkl", 'rb') as handle:
    data_Magribi_MSA_test = pickle.load(handle) 

with open(file_path + "Magribi_MSA-both.pkl", 'rb') as handle:
    data_Magribi_MSA_both = pickle.load(handle) 
    

In [53]:
print(data_English_MSA_both[0:5])
print(data_LAV_MSA_both[0:5])
print(len(data_English_MSA_train))
print(len(data_English_MSA_both))

[['Tom was also there', 'كان توم هنا ايضا'], ['That old woman lives by herself', 'تلك المراة العجوز تسكن بمفردها'], ['He went abroad for the purpose of studying English', 'سافر خارج البلد ليتعلم الانجليزية'], ['There is a fork missing', 'هناك شوكة ناقصة'], ["I don't know this game", 'لا اعرف هذه اللعبة']]
[['لا انا بعرف وحدة راحت ع فرنسا و معا شنتا حطت فيها الفرش', 'لا اعرف واحدة ذهبت الى فرنسا و لها غرفة و ضعت فيها الافرشة'], ['روح بوشك و فتول عاليسار', 'اذهب تقدم و استدر يسارا'], ['لا لا لازم انه يكون عندك موضوع ما في اشي', ' لا لا يجب ان يكون لديك موضوع هذا ضروري'], ['اوعي تبعدي من هون بلاش تضيعي ', 'لا تبتعد عن هنا حتى لا تفقد الطريق '], ['قصدي صراحة يما انا كمان كرهته من يوم ما عملتيه زي ما بتعمله خالتي كرهته و صرت ما باطيقه بالمرة', 'اقصد صراحة يا امي انا ايضا كرهته من يوم حضرته مثلما تحضره خالتي كرهته و اصبحت لا اطيقه ابدا']]
9000
10001


## Prepare Datasets

example: https://iwslt2010.fbk.eu/node/32/

We need to take our training and test sets from the pkl files and create new .txt files that are formatted so that the standard torchtext Dataset class can read them

Data format:
each line consists of three fields divided by the character '\'
sentences consisting of words divided by single spaces
format: <SENTENCE_ID>\<PARAPHRASE_ID>\<TEXT>
Field_1: sentence ID
Field_2: paraphrase ID
Field_3: MT develop sentence / reference translation
Text input example:
DEV_001\01\This is the first develop sentence.
DEV_002\01\This is the second develop sentence.
Reference translation example:
DEV_001\01\1st reference translation for 1st input
DEV_001\02\2nd reference translation for 1st input
...
DEV_002\01\1st reference translation for 2nd input
DEV_002\02\2nd reference translation for 2nd input
...
Languages:
Arabic-English
CSTAR03 testset: 506 sentences, 16 reference translations
IWSLT04 testset: 500 sentences, 16 reference translations
IWSLT05 testset: 506 sentences, 16 reference translations
IWSLT07 testset: 489 sentences, 6 reference translations
IWSLT08 testset: 507 sentences, 16 reference translations
French-English
CSTAR03 testset: 506 sentences, 16 reference translations
IWSLT04 testset: 500 sentences, 16 reference translations
IWSLT05 testset: 506 sentences, 16 reference translations
Turkish-English
CSTAR03 testset: 506 sentences, 16 reference translations
IWSLT04 testset: 500 sentences, 16 reference translations


In [54]:
ls data/AD_NMT-master/

english-Arabic-both.pkl   LAV-MSA-2-both.pkl   Magribi_MSA-both.pkl
english-Arabic-test.pkl   LAV-MSA-2-test.pkl   Magribi_MSA-test.pkl
english-Arabic-train.pkl  LAV-MSA-2-train.pkl  Magribi_MSA-train.pkl


In [0]:
#splits the train dataset into train and validation sets, define test set as datafile
eng_msa_train, eng_msa_val = train_test_split(data_English_MSA_trainval, test_size=.2)
eng_msa_test = data_English_MSA_test

lav_msa_train, lav_msa_val = train_test_split(data_LAV_MSA_trainval, test_size=.2)
lav_msa_test = data_LAV_MSA_test

mag_msa_train, mag_msa_val = train_test_split(data_Magribi_MSA_trainval, test_size=.2)
mag_msa_test = data_Magribi_MSA_test

In [56]:
print(len(eng_msa_train))
print(len(eng_msa_val))

print(len(lav_msa_train))
print(len(lav_msa_val))

print(len(mag_msa_train))
print(len(mag_msa_val))

7200
1800
11044
2761
14188
3548


In [0]:
file_path = 'data/'

def pytorch_format(ds, src='en', trg='msa', datatype=''):
    src_formatted = datatype + '_' + src + '_' + trg + '.' + src
    trg_formatted = datatype + '_' + src + '_' + trg + '.' + trg
    
    with open(file_path + datatype + "/" + src_formatted, 'wt') as srctxt, open(file_path + datatype + "/" + trg_formatted, 'wt') as trgtxt:
        for i, arr in enumerate(ds):
            srctxt.write(datatype.upper() + '_' + str(i).zfill( len(str(len(ds))) - len(str(i))) + '\\01\\' + arr[0] + '\n')
            trgtxt.write(datatype.upper() + '_' + str(i).zfill( len(str(len(ds))) - len(str(i))) + '\\01\\' + arr[1] + '\n')

In [0]:
#splits each language pair file into datasets of single language, to be merged again by the pytorch dataset class later

pytorch_format(eng_msa_train, 'eng', 'msa', 'train')
pytorch_format(eng_msa_val, 'eng', 'msa', 'val')
pytorch_format(eng_msa_test, 'eng', 'msa', 'test')

pytorch_format(lav_msa_train, 'lav', 'msa', 'train')
pytorch_format(lav_msa_val, 'lav', 'msa', 'val')
pytorch_format(lav_msa_test, 'lav', 'msa', 'test')

pytorch_format(mag_msa_train, 'mag', 'msa', 'train')
pytorch_format(mag_msa_val, 'mag', 'msa', 'val')
pytorch_format(mag_msa_test, 'mag', 'msa', 'test')

In [59]:
cat data/train/train_eng_msa.eng

TRAIN_000\01\No one was present
TRAIN_001\01\Tom married a local girl
TRAIN_002\01\Tom is in his room sleeping
TRAIN_003\01\I saw Tom at the library
TRAIN_004\01\I have some pictures to show you
TRAIN_005\01\Just breathe normally
TRAIN_006\01\I need to study
TRAIN_007\01\Did you buy this book?
TRAIN_008\01\I don't know what to do
TRAIN_009\01\The railroad workers are going on strike tomorrow
TRAIN_10\01\I have questions
TRAIN_11\01\Go to work
TRAIN_12\01\No one believes that anymore
TRAIN_13\01\A bunch of people thrust their way toward the rear exit
TRAIN_14\01\Bless you
TRAIN_15\01\You have many books
TRAIN_16\01\I remember his face but I can't remember his name
TRAIN_17\01\Aren't you going to finish your supper?
TRAIN_18\01\Tokyo is the largest city in Japan
TRAIN_19\01\Tom always shouts when he is angry
TRAIN_20\01\She refused his offer
TRAIN_21\01\My father goes to work by bike
TRAIN_22\01\She sat next to me
TRAIN_23\01\Do you know how to use a dictionary?
TRAIN_24\01\I don't know


In [60]:
cat data/train/train_eng_msa.msa

TRAIN_000\01\لم يكن هناك احد
TRAIN_001\01\توم تجوز فتاة محلية
TRAIN_002\01\توم نائم في غرفته
TRAIN_003\01\رايت توم في المكتبة
TRAIN_004\01\لدي بعض الصور لاريك
TRAIN_005\01\فقط تنفس طبيعي
TRAIN_006\01\احتاج الى ان ادرس
TRAIN_007\01\هل اشتريت الكتاب؟
TRAIN_008\01\لا اعرف ما الذي علي فعله
TRAIN_009\01\عمال السكك الحديدية سيضربون عن العمل غدا
TRAIN_10\01\عندي اسئلة
TRAIN_11\01\اذهب الى العمل
TRAIN_12\01\لا احد يصدق ذلك بعد الان
TRAIN_13\01\اندفع حفنة من الناس بطريقهم الى المخرج الخلفي 
TRAIN_14\01\يباركك
TRAIN_15\01\لديك الكثير من الكتب
TRAIN_16\01\اذكر شكله و لكنني لا استطيع ان اتذكر اسمه
TRAIN_17\01\الا ترغب في انهاء عشائك؟
TRAIN_18\01\طوكيو هي اكبر مدينة في اليابان
TRAIN_19\01\توم دائما يصرخ عندما يكون غاضبا
TRAIN_20\01\رفضت عرضه
TRAIN_21\01\يركب ابي الدراجة الى عمله
TRAIN_22\01\جلست بجانبي
TRAIN_23\01\هل تعرف كيف تستخدم معجما؟
TRAIN_24\01\لا ادري
TRAIN_25\01\مدرستنا قريبة من المحطة
TRAIN_26\01\هناك اثنا عشر شهرا في السنة
TRAIN_27\01\من فضلك البس حذاءك
TRAIN_28\01\اخرجت ماري الكعكات من 

In [61]:
cat data/train/train_lav_msa.lav

TRAIN_0000\01\قصدي انون قالولنا بالجامعة ندرسه بالفصل التاني من السنة الاولى مو بالسنة التانية يعني انو التخصص اللي ح اختارو بيزبط معا
TRAIN_0001\01\لازم تحكي المرجع الي جبتي منو المعلومة 
TRAIN_0002\01\وفي طابور الخبز زقفونا وعالفساد عيشونا
TRAIN_0003\01\الحمد لله يا نرجس هيني كنت بحكي مع خالتي مويا قبل شوي و هي بتقولك انه رفيقة بتحاول تتصل فيكي على تليفونك من الصبح بس هو مسكر
TRAIN_0004\01\اشرحلي هاد الكلام
TRAIN_0005\01\لا لانها انصابت بهالضربة
TRAIN_0006\01\بستنى ارائكم
TRAIN_0007\01\بتعرفي شو
TRAIN_0008\01\ليش بتكذبي يا فضيلة
TRAIN_0009\01\السلام عليكم 
TRAIN_010\01\وما بيعرف ان باليوم هاد المحلات بتكون مسكرة 
TRAIN_011\01\بكرة الاربعاء ما بتدرسي
TRAIN_012\01\الكبش بالجنينة
TRAIN_013\01\يعني هو صار شرعي و مشروع
TRAIN_014\01\معهم حق
TRAIN_015\01\قلا احكي و بس انت متل اختي
TRAIN_016\01\قلهم ما بيهمني اذا بدكم تقنعوني جيبولي عالم يجاوبني عن اسئلتي
TRAIN_017\01\جدا
TRAIN_018\01\يا صلاح اذا بدك الشخص صدقني بيقدر يدرس ايمت ما كان
TRAIN_019\01\اذا بدهاتعطيني المصاري و تاخذ هاداك النوكيا 

In [62]:
cat data/train/train_lav_msa.msa

TRAIN_0000\01\ اقصد انهم قالوا لنا في الجامعة ندرسه في السداسي الثاني و ليس في السنة الثانية يعني ان التخصص الذي ساختاره يتماشى معها   
TRAIN_0001\01\انا  فى  احتياج  لمصدر  معلوماتك  لانه
TRAIN_0002\01\وفى  طابور الخبز  قيدونا  وعلى الفساد جعلونا  نعتاده 
TRAIN_0003\01\الحمد لله يا نرجس  لقد كنت اتحدث مع خالتي مويا منذ قليل  و هي تقول لك ان رفيقة تحاول الاتصال بك عبر هاتفك منذ الصباح و لكنه مقفل
TRAIN_0004\01\اشرح لي هذا
TRAIN_0005\01\لا لانها مصابة بتلك الضربة
TRAIN_0006\01\فى انتظار ارائكم
TRAIN_0007\01\اتعلمين ماذا
TRAIN_0008\01\لماذا تكذبين يا فضيلة
TRAIN_0009\01\السلام عليكم 
TRAIN_010\01\وليس يعرف ان هذا النهار المحلات ستكون مغلقة
TRAIN_011\01\ غدا الاربعاء لا تدرسين   
TRAIN_012\01\الكبش في الحديقة
TRAIN_013\01\فهو اذاصار شرعيا و مشروعا
TRAIN_014\01\انهم محقين
TRAIN_015\01\ قال لها تكلمي فحسب انت مثل اختي   
TRAIN_016\01\قال لهم لا يهمني ان اردتم اقناعي فلتاتوني بعالم يجيبني عن اسئلتي
TRAIN_017\01\جدا
TRAIN_018\01\يا صلاح ان اراد الشخص صدقني فانه يستطيع في الدراسة اي وقت
TRAIN_

## Build Vocabulary 

Sentence Piece Google Colab
https://github.com/google/sentencepiece/blob/master/python/sentencepiece_python_module_example.ipynb

In [63]:
!mkdir data/vocab

mkdir: cannot create directory ‘data/vocab’: File exists


In [0]:
"""
Testing:
Create a text file with all the vocab available from all sources for each language for SentencePiece to create a library 

TODO: implement bpemb model for MSA vocab/train SPM on larger datasets for dialects
"""

en_vocab = open("data/vocab/eng_vocab.txt", "wt")
msa_vocab = open("data/vocab/msa_vocab.txt", "wt")
lav_vocab = open("data/vocab/lav_vocab.txt", "wt")
mag_vocab = open("data/vocab/mag_vocab.txt", "wt")

MSA_text = ""
EN_text = ""

def create_vocab(file='', src='en_vocab', tgt='msa_vocab'):
  for line in file:
        src_sent = line[0]
        src_words = src_sent.split(" ")
        for count, word in enumerate(src_words):
            src.write(word)
        src.write("\n")
        
        tgt_sent = line[1]
        tgt_words = tgt_sent.split(" ")
        for count, word in enumerate(tgt_words):
            tgt.write(word)
        tgt.write("\n")

create_vocab(data_English_MSA_both, en_vocab, msa_vocab)
create_vocab(data_LAV_MSA_both, lav_vocab, msa_vocab)
create_vocab(data_Magribi_MSA_both, mag_vocab, msa_vocab)

en_vocab.close()
msa_vocab.close()
lav_vocab.close()
mag_vocab.close()

In [65]:
spm.SentencePieceTrainer.train('--input=data/vocab/eng_vocab.txt --model_prefix=data/eng --vocab_size=2000')
spm.SentencePieceTrainer.train('--input=data/vocab/msa_vocab.txt --model_prefix=data/msa --vocab_size=2000')
spm.SentencePieceTrainer.train('--input=data/vocab/lav_vocab.txt --model_prefix=data/lav --vocab_size=2000')
spm.SentencePieceTrainer.train('--input=data/vocab/mag_vocab.txt --model_prefix=data/mag --vocab_size=2000')


True

In [66]:
sp = spm.SentencePieceProcessor()
sp.load('data/eng.model')

True

In [67]:
ls data

AD_NMT-master/  eng.vocab  lav.vocab  mag.vocab  msa.vocab  train/  vocab/
eng.model       lav.model  mag.model  msa.model  test/      val/


In [68]:
print(sp.encode_as_pieces('This is a test'))
print(sp.encode_as_ids('This is a test'))

['▁This', '▁', 'is', '▁', 'a', '▁', 't', 'est']
[89, 83, 12, 83, 8, 83, 6, 309]


## Spacy Tokenizer

In [0]:
eng_vocab_data = open("data/eng.vocab", "r")
msa_vocab_data = open("data/msa.vocab", "r")
lav_vocab_data = open("data/lav.vocab", "r")
mag_vocab_data = open("data/mag.vocab", "r")

eng_vocab_list = []
msa_vocab_list = []
lav_vocab_list = []
mag_vocab_list = []

for line in eng_vocab_data.readlines():
    eng_vocab_list.append(line.split("\t")[0])

for line in msa_vocab_data.readlines():
    msa_vocab_list.append(line.split("\t")[0])

for line in lav_vocab_data.readlines():
    lav_vocab_list.append(line.split("\t")[0])

for line in mag_vocab_data.readlines():
    mag_vocab_list.append(line.split("\t")[0])


In [70]:
msa_vocab_list[0:200]

['<unk>',
 '<s>',
 '</s>',
 '▁',
 'ا',
 'ت',
 'ي',
 'و',
 'ال',
 'م',
 'ك',
 'ب',
 'ه',
 'ة',
 'ل',
 'ن',
 'ان',
 'ر',
 'في',
 '؟',
 'ف',
 'س',
 'ها',
 'ع',
 'من',
 'ين',
 'د',
 'ون',
 'ح',
 'نا',
 'وا',
 'لا',
 'ى',
 'ق',
 'هم',
 'ج',
 'ما',
 'ش',
 'ط',
 'على',
 'ز',
 'الم',
 '▁نعم',
 'ني',
 'هذا',
 'انت',
 'لي',
 'هو',
 '▁و',
 'فيال',
 'ست',
 'يا',
 'انا',
 'عن',
 'ار',
 'ص',
 '▁ا',
 'ض',
 'سي',
 '▁لا',
 'خ',
 'غ',
 'ذلك',
 'سا',
 'اني',
 'ية',
 'كل',
 'انه',
 'مع',
 'ث',
 'ات',
 'منال',
 'هناك',
 'ذهب',
 'اء',
 'هل',
 '▁انا',
 'الى',
 'بال',
 'ام',
 'لك',
 'ذ',
 '▁هل',
 'عمل',
 'اع',
 'ماذا',
 'اذا',
 'لكن',
 'الان',
 'كان',
 'جد',
 'اد',
 'اب',
 'ائ',
 'ته',
 'او',
 'ظ',
 'فى',
 'لات',
 'حتى',
 'عندما',
 'فقط',
 'لل',
 'توم',
 'جدا',
 'كم',
 'لاي',
 'هنا',
 'انها',
 'بعد',
 'هي',
 'لم',
 'لما',
 'درس',
 '▁لكن',
 'فعل',
 'وال',
 'اح',
 'الا',
 'شيء',
 'قال',
 'اليوم',
 'عد',
 'حب',
 'به',
 'كانت',
 'ايضا',
 'قل',
 'مثل',
 'شيئا',
 '▁هذا',
 'اج',
 'اخ',
 'را',
 'ئ',
 'يد',
 'لان',
 '

In [0]:
en_vocab = Vocab(strings=eng_vocab_list)
spacy_en_tokenizer = Tokenizer(en_vocab)

msa_vocab = Vocab(strings=msa_vocab_list)
spacy_msa_tokenizer = Tokenizer(msa_vocab)

lav_vocab = Vocab(strings=lav_vocab_list)
spacy_lav_tokenizer = Tokenizer(lav_vocab)

mag_vocab = Vocab(strings=mag_vocab_list)
spacy_mag_tokenizer = Tokenizer(mag_vocab)

## TF Tokenizer

https://huggingface.co/transformers/model_doc/t5.html#t5tokenizer

In [0]:
ls data/

In [0]:
msa_tokenizer = T5Tokenizer("data/msa.model")
en_tokenizer = T5Tokenizer("data/en.model")

In [0]:
input_ids = msa_tokenizer.encode('هل بامكاني الدخول؟ </s>', return_tensors='pt')

In [0]:
input_ids

In [0]:
#decode to make sure you can go back and forth between the encoding properly 
#@raef can you verify the Arabic?

In [0]:
ls data/train

## Pytorch Data Set and Data Loader

https://github.com/google-research/text-to-text-transfer-transformer

pytorch dataset: https://pytorch.org/text/_modules/torchtext/datasets/translation.html

pytorch dataset documentation: https://torchtext.readthedocs.io/en/latest/datasets.html#iwslt

example dataset: https://iwslt2010.fbk.eu/node/32/

Field API: https://pytorch.org/text/data.html#torchtext.data.Field
Field Source: https://pytorch.org/text/_modules/torchtext/data/field.html#Field

https://pytorch.org/text/_modules/torchtext/data/utils.html#get_tokenizer
    
we need to pass our tokenizer as a function     

In [0]:
SRC = Field(tokenize = spacy_en_tokenizer,
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = False)

TRG = Field(tokenize = spacy_msa_tokenizer,
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = False)

In [0]:
train_dataset = TranslationDataset(path='data/train/train.', exts=('en', 'msa'), fields=(SRC, TRG))

In [0]:
SRC.build_vocab(dataset)
TRG.build_vocab(dataset)

In [0]:
ex = dataset[0]

In [0]:
print(ex.src) 
print(ex.trg)

TRAIN_000\01\They are proud of their daughter
TRAIN_0000\01\هذه الرسالة منقولة يا ايمان


In [0]:
ls data/train

train.en  train.lav  train.msa


In [0]:
#train, validation and test are probably wrong.  
#https://github.com/pytorch/text/blob/master/torchtext/datasets/translation.py

'''
Arguments:
            exts: A tuple containing the extension to path for each language.
            fields: A tuple containing the fields that will be used for data
                in each language.
            path (str): Common prefix of the splits' file paths, or None to use
                the result of cls.download(root).
            root: Root dataset storage directory. Default is '.data'.
            train: The prefix of the train data. Default: 'train'.
            validation: The prefix of the validation data. Default: 'val'.
            test: The prefix of the test data. Default: 'test'.
            Remaining keyword arguments: Passed to the splits method of
                Dataset.

'''


train_data, valid_data, test_data = dataset.splits(path= 'data/', train='train/train', validation='val/val', test='test/test', exts=('.en', '.msa'),
                                                    fields = (SRC, TRG))

In [0]:
BATCH_SIZE =32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device)

## Model Training

https://huggingface.co/transformers/model_doc/t5.html#training

In [0]:
configuration = T5Config()

In [0]:
model = T5PreTrainedModel.from_pretrained('t5-base')

In [0]:
model
model.init_weights()
optimizer = optim.AdamW(model.parameters())
pad_idx = TRG.vocab.stoi['<pad>']
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

0


ValueError: ignored

In [0]:
def train(model:nn.Module, 
          iterator:BucketIterator, 
          optimizer: optim.Optimizer,
          criterion:nnCrossEntropyLoss, 
          clip: float):
  model.train()
  epoch_loss = 0
  for i, batch in enumerate(train_iterator):
    src = batch.src
    trg = batch.trg

    optimizer.zero_grad()

    output = model(src, trg)
    loss = criterion(output[1:].view(-1, output.shape[2]), trg[1:].view(-1))
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
    optimizer.step()
    epoch_loss += loss.item()
  return epoch_loss / len(iterator)

In [0]:
def evaluate(model: nn.Module,
             iterator: BucketIterator,
             criterion: nn.Module):

    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for _, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg
            output = model(src, trg, 0) #turn off teacher forcing
            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(iterator)

In [0]:
import time

In [0]:
def epoch_time(start_time: int,
               end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [0]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

## Model Inference

https://github.com/huggingface/transformers/blob/master/examples/translation/t5/evaluate_wmt.py